## Assignement 2: Exploritory Data Analysis

William "Taylor" Martinez

#### Notes and Setup

[`dplyr` to `polars`](https://docs.pola.rs/user-guide/migration/pandas/#column-assignment):
| Operation                | Syntax                                   |
|--------------------------|------------------------------------------|
| read (lazy)              | `df.scan_csv()` or `df.scan_parquet()`  |
| collect                  | `df.collect()`                           |
| select                   | `df.select("col_name1", "col_name2")`   |
| filter                   | `df.filter(pl.col("col_name") < 10)`    |
| mutate                   | `df.with_columns(new_col_name = pl.col("col_name") * 10)` |
| mutate (conditional)   | ```df.with_columns( pl.when(pl.col("c") == 2) .then(pl.col("b")) .otherwise(pl.col("a")).alias("a") )``` |
| missing                  | `null`                                   |
| all columns               | `pl.all()`                                  |

`csv` vs `parquet`:
    Parquet was chosen over `csv` because it takes up less space, it is columnar formatted, and is has improved query performance. [medium](https://medium.com/@dinesh1.chopra/unveiling-the-battle-apache-parquet-vs-csv-exploring-the-pros-and-cons-of-data-formats-b6bfd8e43107)








In [1]:
import polars as pl # Used for Lazy, Parallel Queries 
import pandas as pd # Compatibility
from great_tables import GT, md, html # Table Generation
import sidetable # Table Generation. It is used but as stb idk why
import os


data_path = "../../Data/" # Relative Path to Data Folder.
figures_path = "./figures_taylor/" # Relative Path to Figures.

In [2]:
df = pl.scan_csv(data_path + "heart_2022_with_nans.csv")
df.collect().write_parquet(data_path + "heart_2022_with_nans.parquet")
df = pl.scan_parquet(data_path + "heart_2022_with_nans.parquet")

### EDA Task 1: Create `HadHeartDisease` column

1. Set `HadHeartDisease` to `True` if the survey participant reported having a least one of the following adverse cardiovascular events: heart attack (`HadHeartAttack`), stroke (`HadStroke`), or angina  (`HadAngina`).

In [3]:
### Create HadHeartDisease Column
df = df.with_columns(
    pl.when(
        (pl.col("HadHeartAttack") == "Yes") |
        (pl.col("HadStroke") == "Yes") |
        (pl.col("HadAngina") == "Yes")
    )
    .then(pl.lit("Yes"))
    .otherwise(pl.lit("No"))
    .alias("HadHeartDisease")
)

df.fetch(5)

State,Sex,GeneralHealth,PhysicalHealthDays,MentalHealthDays,LastCheckupTime,PhysicalActivities,SleepHours,RemovedTeeth,HadHeartAttack,HadAngina,HadStroke,HadAsthma,HadSkinCancer,HadCOPD,HadDepressiveDisorder,HadKidneyDisease,HadArthritis,HadDiabetes,DeafOrHardOfHearing,BlindOrVisionDifficulty,DifficultyConcentrating,DifficultyWalking,DifficultyDressingBathing,DifficultyErrands,SmokerStatus,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AgeCategory,HeightInMeters,WeightInKilograms,BMI,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos,HadHeartDisease
str,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,str
"""Alabama""","""Female""","""Very good""",0.0,0.0,"""Within past ye…","""No""",8.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Not at all (ri…","""No""","""White only, No…","""Age 80 or olde…",null,null,null,"""No""","""No""","""Yes""","""No""","""Yes, received …","""No""","""No""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,null,"""No""",6.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 80 or olde…",1.6,68.04,26.57,"""No""","""No""","""No""","""No""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Very good""",2.0,3.0,"""Within past ye…","""Yes""",5.0,null,"""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""No""","""White only, No…","""Age 55 to 59""",1.57,63.5,25.61,"""No""","""No""","""No""","""No""",null,"""No""","""Yes""","""No"""
"""Alabama""","""Female""","""Excellent""",0.0,0.0,"""Within past ye…","""Yes""",7.0,null,"""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""Yes""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Current smoker…","""Never used e-c…","""Yes""","""White only, No…",null,1.65,63.5,23.3,"""No""","""No""","""Yes""","""Yes""","""No, did not re…","""No""","""No""","""No"""
"""Alabama""","""Female""","""Fair""",2.0,0.0,"""Within past ye…","""Yes""",9.0,null,"""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""No""","""Never smoked""","""Never used e-c…","""Yes""","""White only, No…","""Age 40 to 44""",1.57,53.98,21.77,"""Yes""","""No""","""No""","""Yes""","""No, did not re…","""No""","""No""","""No"""


### EDA Task 2: Drop Observations With Too Many Missing Values

1. Create `df_heart_drop` where participants are dropped if Heart attack (`HadHeartAttack`), stroke (`HadStroke`), or angina  (`HadAngina`) are missing.

2. From `df_heart_drop`, make multiple dataframes that drop survey participants based
on the number of missing responses.

3. Collect the dataframes and return the length of each entry.


In [9]:
### Drop observations where HadHeartAttack, HadStroke, and HadAngina are missing

df_heart_drop = (df.drop_nulls(
    subset=["HadHeartAttack", "HadStroke", "HadAngina"]
    ))

# Create a dictionary to keep track of all lazy dataframes
df_dict = {"df" : df, "df_heart_drop" : df_heart_drop}

# If number of missing is > threshold, drop the observation.
thresholds = [0, 1, 3, 5, 10, 20, 40]

for threshold in thresholds:
    df_name = f"df_heart_drop_{threshold:02}" # f-string
    # Drop observation if # of null values is greater than threshold and 
    # Add to dictionary
    df_dict[df_name] = df_heart_drop.filter(
        pl.sum_horizontal(pl.all().is_null()) <= threshold
        )
    # Print the number of rows in each dataframe.
    print(df_name + ": " + str(
        df_dict[df_name].select(pl.len()).collect().item()
        ))

df_heart_drop_00: 246022
df_heart_drop_01: 331181
df_heart_drop_03: 381718
df_heart_drop_05: 391725
df_heart_drop_10: 410245
df_heart_drop_20: 436507
df_heart_drop_40: 437510


### EDA Task 3: Impute remaining missing values

1. Show column types

2. Impute float and integer values by median.

3. Impute string values by mode.

Note: This is applied to df_heart_drop, other dataframes can be imputed.


In [8]:
### Unique data types:
# print(set(df_heart_drop.dtypes))

### Imputation
def impute_df(df):
    df = df.collect() # Collect because iteration is needed.
    for i in range(len(df.columns)):
        col_name = df.columns[i]
        dtype = df.dtypes[i]
        ## Impute string using the mode
        if dtype == pl.Utf8:
            mode_value = df[col_name].mode()
            df = df.with_columns(df[col_name].fill_null(mode_value))
        ## Impute float using the median
        elif dtype == pl.Float64:
            median_value = df[col_name].median()
            df = df.with_columns(df[col_name].fill_null(median_value))
        ## Warning catch: if type is not a string or float.
        else:
            print("Unexpected type:", dtype)
    return df

### Show the number of missing values for each column
# impute_df(df_heart_drop).null_count()
df_dict_imp = dict()
for key, value in df_dict.items():
    df_name = f"{key}_imp"
    df_dict_imp[df_name] = pl.LazyFrame(impute_df(value))


In [14]:
"""
Data Dictionaries
"""
df_dict # Data with no imputation

df_dict_imp # Data with imputation

df_dict_all = dict()
df_dict_all.update(df_dict)
df_dict_all.update(df_dict_imp)
# print(len(df_dict_all)) # 18 dataframes
print(df_dict_all) # Data with and without imputation

"""
Save to Google Drive. Create a shortcut of your decisiontreebruining
from the shared drive your personal drive (Online). Then download Google Drive 
Desktop and create a sys link using `ln -s <source path> <linked folder path>`.
"""
for name, df in df_dict_all.items():
    df.collect().write_parquet(f"{data_path}GoogleDrive/{name}.parquet")


def lazy_read_parquet(path):
    """
    Lazy read all parquet files in a folder.
    ---
    Args: Relative Path to folder
    Return: Dictionary of lazy dataframes
    """
    lazy_frames_dict = {}
    for filename in os.listdir(path): # Iterate over each file
        if filename.endswith(".parquet"):
            file_path = os.path.join(path, filename) # Path for polars
            lazy_frame = pl.scan_parquet(file_path) # Lazy read
            key = os.path.splitext(filename)[0] # key = filename
            lazy_frames_dict[key] = lazy_frame # dictionary
    return lazy_frames_dict


{'df': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x146A791B0>, 'df_heart_drop': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x3EF6F6290>, 'df_heart_drop_00': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x3EF6F6E60>, 'df_heart_drop_01': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x3EF457EE0>, 'df_heart_drop_03': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x3EF457FA0>, 'df_heart_drop_05': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x1080FB460>, 'df_heart_drop_10': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x3EF0F9150>, 'df_heart_drop_20': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x3EF6F5060>, 'df_heart_drop_40': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": String}] at 0x3EF6F4AC0>, 'df_imp': <LazyFrame [41 cols, {"State": String … "HadHeartDisease": St

### EDA Task 4: Show tables/graphs representing each task

1. Table comparing the number of observations of `HadHeartAttack`, `HadStroke`, `HadAngina` and the summarized column `HadHeartDisease`.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

2. Table showing the number of observations after setting a missing threshold.
    a. Provide an example using a rudimentary model (Logit Regression with enet regularization)

3. Table comparing using imputation.

In [7]:
### Table 1: Comparing `HadHeartAttack`, `HadStroke`, `HadAngina` to `HadHeartDisease`
tbl_1 = (df.collect().to_pandas().stb.freq(
    ["HadHeartAttack", "HadStroke", "HadAngina", "HadHeartDisease"])
    .drop(columns=['cumulative_count', 'cumulative_percent'])
)

tbl_1["percent"] = tbl_1["percent"] / 100

tbl_event_freq = (GT(tbl_1)
    .tab_header(title="Adverse Cardiovascular Event Frequencies")
    .cols_label(
        count=html("Count"),  # Centering text using HTML style attribute
        percent=html("Percent")
    )
    .cols_align(align="center")
    .fmt_percent(columns="percent")
    .tab_source_note(
        source_note="• Missing values from HadHeartAttack, HadStroke, and HadAngina were dropped."
    )
)

"""
To save the table, install selenium using `pip install -U selenium` then
download chromedriver and put into `usr/local/bin`.
"""
# tbl_event_freq.save(file=f"{figures_path}tbl_event_freq.png")
tbl_event_freq

GT(_tbl_data=  HadHeartAttack HadStroke HadAngina HadHeartDisease   count   percent
0             No        No        No              No  387696  0.886142
1             No        No       Yes             Yes   12438  0.028429
2             No       Yes        No             Yes   11939  0.027289
3            Yes        No        No             Yes    9789  0.022374
4            Yes        No       Yes             Yes    9259  0.021163
5            Yes       Yes       Yes             Yes    2568  0.005870
6            Yes       Yes        No             Yes    2091  0.004779
7             No       Yes       Yes             Yes    1730  0.003954, _body=<great_tables._gt_data.Body object at 0x146abcd00>, _boxhead=Boxhead([ColInfo(var='HadHeartAttack', type=<ColInfoTypeEnum.default: 1>, column_label='HadHeartAttack', column_align='center', column_width=None), ColInfo(var='HadStroke', type=<ColInfoTypeEnum.default: 1>, column_label='HadStroke', column_align='center', column_width=None), ColInfo(var='HadAngina', type=<ColInfoTypeEnum.default: 1>, column_label='HadAngina', column_align='center', column_width=None), ColInfo(var='HadHeartDisease', type=<ColInfoTypeEnum.default: 1>, column_label='HadHeartDisease', column_align='center', column_width=None), ColInfo(var='count', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Count', type='html'), column_align='center', column_width=None), ColInfo(var='percent', type=<ColInfoTypeEnum.default: 1>, column_label=Text(text='Percent', type='html'), column_align='center', column_width=None)]), _stub=Stub([RowInfo(rownum_i=0, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=1, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=2, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=3, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=4, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=5, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=6, group_id=None, rowname=None, group_label=None, built=False), RowInfo(rownum_i=7, group_id=None, rowname=None, group_label=None, built=False)]), _row_groups=[], _group_rows=GroupRows([]), _spanners=Spanners([]), _heading=Heading(title='Adverse Cardiovascular Event Frequencies', subtitle=None, preheader=None), _stubhead=None, _source_notes=['• Missing values from HadHeartAttack, HadStroke, and HadAngina were dropped.'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x146abdb70>, _formats=[<great_tables._gt_data.FormatInfo object at 0x146abd630>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_in